[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_hedge_ratio/EMQA_hedge_ratio.ipynb)

# EMQA_hedge_ratio
Optimal hedge ratio via OLS regression of Brent on WTI returns.
**Output:** `hedge_ratio.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
# Fetch Brent and WTI
brent = fetch('BZ=F')
wti = fetch('CL=F')

# Align and compute returns
common = brent.index.intersection(wti.index)
brent = brent.loc[common]
wti = wti.loc[common]
ret_b = np.log(brent / brent.shift(1)).dropna()
ret_w = np.log(wti / wti.shift(1)).dropna()
common2 = ret_b.index.intersection(ret_w.index)
ret_b = ret_b.loc[common2]
ret_w = ret_w.loc[common2]

# OLS hedge ratio
from numpy.polynomial.polynomial import polyfit
coeffs = np.polyfit(ret_w.values, ret_b.values, 1)
hedge_ratio = coeffs[0]
intercept = coeffs[1]
print(f"Optimal hedge ratio (beta): {hedge_ratio:.4f}")
print(f"Intercept: {intercept:.6f}")

In [ ]:
# 2-panel: Scatter + regression, Portfolio variance vs hedge ratio
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Panel 1 - Scatter + regression line
axes[0].scatter(ret_w.values, ret_b.values, alpha=0.3, s=8, color=COLORS['blue'])
x_line = np.linspace(ret_w.min(), ret_w.max(), 100)
axes[0].plot(x_line, hedge_ratio * x_line + intercept, color=COLORS['red'], linewidth=2,
             label=f'h* = {hedge_ratio:.3f}')
axes[0].set_xlabel('WTI Returns')
axes[0].set_ylabel('Brent Returns')
axes[0].set_title('OLS Regression: Brent on WTI')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False)

# Panel 2 - Portfolio variance vs hedge ratio
h_range = np.linspace(0, 2, 200)
var_b = ret_b.var()
var_w = ret_w.var()
cov_bw = np.cov(ret_b.values, ret_w.values)[0, 1]
port_var = var_b - 2 * h_range * cov_bw + h_range**2 * var_w
min_h = cov_bw / var_w
axes[1].plot(h_range, port_var, color=COLORS['blue'], linewidth=2)
axes[1].axvline(min_h, color=COLORS['red'], linestyle='--', linewidth=1.5, label=f'h* = {min_h:.3f}')
axes[1].scatter([min_h], [port_var[np.argmin(np.abs(h_range - min_h))]], color=COLORS['red'], s=80, zorder=5)
axes[1].set_xlabel('Hedge Ratio (h)')
axes[1].set_ylabel('Portfolio Variance')
axes[1].set_title('Hedged Portfolio Variance vs Hedge Ratio')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False)

plt.tight_layout()
save_fig(fig, 'hedge_ratio.pdf')
plt.show()